In [ ]:
import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

driver = webdriver.Chrome("C:\\Users\\iris8\\Downloads\\chromedriver_win32\\chromedriver.exe")
driver2 = webdriver.Chrome("C:\\Users\\iris8\\Downloads\\chromedriver_win32\\chromedriver.exe")


driver.get("https://ebook.kyobobook.co.kr/dig/pnd/showcase?pageNo=323&cmdt=EBK&clst1=01&clst2=&clst3=&landing=Y")


# 카테고리 추출
driver.find_elements(By.CSS_SELECTOR, '.btn_sub_depth')[1].click()
time.sleep(2)
datas = driver.find_elements(By.CSS_SELECTOR, '.sub_depth_box')


categories = []

for data in datas :
    li = data.find_elements(By.CSS_SELECTOR, '.sub_depth_list a')
    for i in range(len(li)) :
        if li[i].text != '' and li[i].text != '섹슈얼로맨스' and li[i].text != '웹툰' and li[i].text != '로맨스' and li[i].text != 'BL':
            categories.append(li[i].text)

# for i in range(len(categories)) :
#     print(categories[i])

# 이어서 하기
categories = categories[17:]


# 카테고리 별
for category in categories :
    # print(category)
    isbns = []
    fields = []
    books = []
    driver.find_elements(By.CSS_SELECTOR, '.btn_sub_depth')[1].click()
    time.sleep(2)
    driver.find_element(By.LINK_TEXT, category).click()


    # 10페이지 씩
    for i in range(11) :
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        data = soup.select('.prodDt_detail')
        contents = driver.find_elements(By.CSS_SELECTOR, '.prodDt')

        # 상세페이지 주소를 통한 isbn & 카테고리
        for link in data :
            try :
                detail = link.select_one('strong h3 a').attrs['href']
                driver2.get(detail)
                time.sleep(2)
                isbn = driver2.find_elements(By.CSS_SELECTOR, '.prod_pordInfo_box em')[1].text
                field = driver2.find_element(By.CSS_SELECTOR, '.intro_category_link').text
                isbns.append(isbn)
                fields.append(field)
            except :
                isbns.append('0000000000000')
                fields.append(category)

        # 전체 목록에서 그 외 정보 추출
        for content in contents :
            try :
                img = content.find_element(By.CSS_SELECTOR, '.prodDt_cover img').get_attribute('src')
            except :
                img = ''
            
            try :
                title = content.find_element(By.CSS_SELECTOR, '.prodDt_detail strong h3').text
            except:    
                title = ''
            
            try: 
                author = content.find_element(By.CSS_SELECTOR, '.prodDt_detail .prodDt_info').text.split(' ')[0]
            except:
                author = ''
            
            try :
                price = content.find_element(By.CSS_SELECTOR, '.prodDt_detail .prodDt_price b').text
            except :
                price = '0'
            
            try :
                star = content.find_element(By.CSS_SELECTOR, '.prodDt_detail .prodDt_review b').text
            except:
                star = "0.0"

            price = price.replace(',', '')
            books.append([title, author, price, star, img])

        # 다음 페이지 이동
        try :
            driver.find_element(By.CSS_SELECTOR, '.btn_page.next').click()
        except :
            pass
        


# 데이터 프레임 생성
    df = pd.DataFrame(columns=['ISBN', 'title', 'author', 'price', 'star', 'category', 'img'])

    for i in range(len(isbns)) :
        df.loc[i] = {
            "ISBN" : isbns[i],
            "title" : books[i][0],
            "author" : books[i][1],
            "price" : int(books[i][2]),
            "star" : books[i][3],
            "category" : fields[i],
            "img" : books[i][4],
        }
    cat_name = category.replace('/', ',')
    df.to_csv('./res/'+cat_name+'.csv', index=False, encoding='cp949')